<h1> This is a Data Science Capstone Project in Coursera </h1>
<h3> (for the IBM Data Science Professional Certificate) </h3>

In [1]:
import pandas as pd
import numpy as np
import json
import requests
!pip install bs4
from bs4 import BeautifulSoup

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


<h2>Q1 of week 3 Assignment</h2>

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url).text
soup = BeautifulSoup(results, 'html.parser')

In [4]:
table = soup.find_all('table')[0]
alltd = table.find_all('td')
print(alltd[0], '\n', alltd[2], '\n', alltd[20])

<td style="width:11%; vertical-align:top; color:#ccc;">
<p><b>M1A</b><br/><span style="font-size:85%;"><i>Not assigned</i></span>
</p>
</td> 
 <td style="width:11%; vertical-align:top;">
<p><b>M3A</b><br/><span style="font-size:85%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>)</span>
</p>
</td> 
 <td style="vertical-align:top;">
<p><b>M3C</b><br/><span style="font-size:85%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Don_Mills" title="Don Mills">Don Mills</a>)<br/>South<br/>(<a href="/wiki/Flemingdon_Park" title="Flemingdon Park">Flemingdon Park</a>)</span>
</p>
</td>


In [5]:
table_contents = []

for td in alltd:
    cell = {}
    if td.span.text == 'Not assigned':
        pass
    else:
        cell['PostalCode'] = td.p.b.text
        temp = td.span.text.split('(')
        cell['Borough'] = temp[0]
        cell['Neighborhood'] = (((temp[1].strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df = pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade' : 'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern' : 'East Toronto Business',
                                             'EtobicokeNorthwest' : 'Etobicoke Northwest',
                                             'East YorkEast Toronto' : 'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre' : 'Mississauga'})

df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [6]:
df.shape

(103, 3)

<h2> Q2 of Week 3 Assignment </h2>

In [15]:
!pip install geocoder
import geocoder

In [49]:
#since geocoder does not work, the geospatial csv file is used
df_geo = pd.read_csv('Geospatial_Coordinates.csv')
df_geo.rename({'Postal Code' : 'PostalCode'}, axis=1, inplace=True)
df2 = pd.merge(df, df_geo, how='inner', on='PostalCode')
df2


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


<h2> Q3 of Week 3 Assignment </h2>

<h3> To plot clusters of boroughs of Toronto containing the word 'Toronto'</h3>

In [96]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

Solving environment: done

# All requested packages already installed.



In [97]:
df2['Interest'] = df2['Borough'].str.contains('Toronto')

#there are 7 boroughs (clusters)
df3 = df2[df2['Interest']==True]
df3.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Interest
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,True
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,True
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,True
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,True
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,True


In [98]:
print('Shape of df3: ', df3.shape)
print('Number of unique boroughs:', len(df3['Borough'].unique()))

Shape of df3:  (39, 6)
Number of unique boroughs: 7


In [106]:
# Coordinates of Toronto from Google
tor_lat, tor_lng = 43.7181552, -79.5184859


'#2c7ef7'

In [108]:
# create map
map_clusters = folium.Map(location=[tor_lat, tor_lng], zoom_start=11)

# set color scheme for the clusters and map a borough to a color
colors_array = cm.rainbow(np.linspace(0, 1, 7))
rainbow = [colors.rgb2hex(i) for i in colors_array]
uqb = df3['Borough'].unique()
coldict = dict(zip(uqb, rainbow))

# add markers to the map
markers_colors = []

for lat, lng, bor, neigh in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighborhood']):
    label = folium.Popup(str(neigh) + ' of ' + str(bor), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=coldict[bor],
        fill=True,
        fill_color=coldict[bor],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters